In [1]:
#모듈 설치
!pip install folium
!pip install pyproj

In [2]:
#모듈 임포트
import pandas as pd
import folium
import itertools
import json
from folium import Marker
from folium.plugins import MarkerCluster
from pyproj import Transformer, transform, Proj
from shapely.geometry import Point, Polygon

In [3]:
#지도 보여줄 좌표 인하대 기준으로 지정
map = folium.Map(location=[37.451023, 126.655486], zoom_start=11)

In [4]:
#원하는 데이터의 좌표 로드
df = pd.read_csv('전체숙박업소.csv', encoding='utf-8')

In [5]:
proj_UTMK = Proj(init='epsg:5178') # UTM-K(Bassel) 도로명주소 지도 사용 중

# WGS1984
proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표
transformer_5178_4326 = Transformer.from_proj(proj_UTMK, proj_WGS84) # 도로명주소 기준으로 transform

p_4326 = (126.624176, 37.433297) # 격자가 미추홀구 전지역 보여주기 위한 시작 좌표
meters = 500 # 500m 간격으로 격자 생성
n = 13 # 13*13의 격자 생성

p_x_5178, p_y_5178 = transform(proj_WGS84,proj_UTMK , p_4326[0], p_4326[1])

geo_jsons = {"type": "FeatureCollection","features":[]}
dic = {} # 폴리건을 만들기 위한 dict "n/n":[(좌표),(좌표),(좌표),(좌표)] 구조 
for x, y in itertools.product(range(n), range(n)): # 가로 13번, 세로 13번 격자 생성
    grid_lower_l = transformer_5178_4326.transform(p_x_5178 + (x + 0) * meters, p_y_5178 + (y + 0) * meters)
    grid_lower_r = transformer_5178_4326.transform(p_x_5178 + (x + 1) * meters, p_y_5178 + (y + 0) * meters)
    grid_upper_r = transformer_5178_4326.transform(p_x_5178 + (x + 1) * meters, p_y_5178 + (y + 1) * meters)
    grid_upper_l = transformer_5178_4326.transform(p_x_5178 + (x + 0) * meters, p_y_5178 + (y + 1) * meters)
    dic[str(x)+"/"+str(y)] = [(grid_lower_l[1],grid_lower_l[0]),(grid_lower_r[1],grid_lower_r[0]),(grid_upper_r[1],grid_upper_r[0]),(grid_upper_l[1],grid_upper_l[0])]

    geo_json =  {
                "type": "Feature",
                "properties":{"code": str(x)+"/"+str(y)},
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            grid_lower_l,
                            grid_lower_r,
                            grid_upper_r,
                            grid_upper_l,
                            grid_lower_l
                        ]
                    ]
                }
            }
    geo_jsons["features"].append(geo_json)

C:\Users\hybda\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\hybda\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\hybda\AppData\Local\Temp/ipykernel_3312/2376824063.py:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  p_x_5178, p_y

In [7]:
polygonDic ={}
for key in dic.keys():
  engineer=dic[key]
  engineer_poly = Polygon(engineer)
  polygonDic[key] = engineer_poly
#polygonDic에 저장된 폴리건 중에 포함되는 걸 찾기 
def pl(df):
  test_code1 = Point(df["위도"], df["경도"])
  for key in polygonDic.keys():
    engineer_poly=polygonDic[key]
    if test_code1.within(engineer_poly) == True :
      return key
df["polygon"] = df.apply(pl,axis=1)
#나누어둔 구역에 매칭할 구역별 세대수  "x/y"로 매칭 (0~13/0~13) 
apartData = df.groupby("polygon").sum()["경쟁업체 개수"]
apartData

KeyError: '전체숙박업소'

In [8]:
# 격자 지도에 추가
folium.GeoJson(
    geo_jsons
).add_to(map)
map

In [9]:
# 원하는 데이터좌표 격자에 비율로 표시
map.choropleth(geo_data=geo_jsons,
             data=apartData, 
             fill_color='PuRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.code',
            )

NameError: name 'apartData' is not defined

In [ ]:
# 원하는 데이터좌표 격자에 점으로 표시
for i in df[["위도","경도"]].values:
  folium.Circle(location=[i[0],i[1]],radius=1).add_to(map)
map